# supernodes

The procedure of combining public transportation stops into supernodes.

Requires:
1. `data/preprocessed/stops.json`
2. `data/preprocessed/routes.json`

Produces:
1. `data/supernodes/supernode_threshold_metrics.json`
2. `data/supernodes/supernodes.json`

Alters:
1. `data/preprocessed/stops.json` (adds stop to supernode mapping)
2. `data/preprocessed/routes.json` (adds stops to supernodes mapping)

In [1]:
import sys
sys.path.append('..')

In [2]:
from typing import Set, Tuple, List

import numpy as np
import pandas as pd
import networkx as nx
import seaborn as sns
from cartopy import crs as ccrs
from tqdm import tqdm
from matplotlib import pyplot as plt
from matplotlib.patches import Circle

from myutils.spatial import get_earth_distances
from myutils.json import save_json
from myutils.matplotlib import matplotlib_latex, matplotlib_seaborn_style

from config import stops_fpath, routes_fpath, supernode_threshold_metrics_fpath, \
    supernodes_fpath
from cartopy_config import vaska_area, get_node_props, supernode_color, supernode_zorder
from ptn.cartopy import add_map_subplot

matplotlib_seaborn_style()
matplotlib_latex()

In [3]:
stops = pd.read_json(stops_fpath, encoding='utf-8').set_index('id')

stops.head(2)

,type,name,lat,lon,diameter,supernode
id,,,,,,
16396,tram,"Конечная станция ""Река Оккервиль""",59.908881,30.494671,0.0,620
16367,tram,Пр. Солидарности,59.911857,30.499882,0.0,827


In [4]:
coords = stops[['lat', 'lon']]

coords.head(2)

,lat,lon
id,,
16396,59.908881,30.494671
16367,59.911857,30.499882


In [5]:
diameters = stops['diameter'].values
shifts = (diameters.reshape(-1, 1) + diameters.reshape(1, -1)) / 2

distances = get_earth_distances(coords.values, coords.values) - shifts

distances.shape

(7661, 7661)

In [6]:
def build_supernodes(th: float) -> pd.DataFrame:
    closeness_adj_mx = (distances <= th).astype(int)
    closeness_graph = nx.from_numpy_array(closeness_adj_mx)

    id2stop_id = dict(enumerate(stops.index))
    
    closeness_graph = nx.relabel_nodes(closeness_graph, id2stop_id)

    supernodes = sorted(nx.connected_components(closeness_graph), key=len, reverse=True)
    
    def supernode_diameter(supernode: Set[int]) -> float:
        coords_ = coords.loc[supernode, ['lat', 'lon']].values
        distances_ = get_earth_distances(coords_, coords_)
        diameter = distances_.max()

        return diameter

    supernode_diameters = [supernode_diameter(supernode) for supernode in supernodes]
    
    supernodes_df = []

    for i, supernode in enumerate(supernodes):
        lat, lon = coords.loc[supernode, ['lat', 'lon']].mean()
        diameter = supernode_diameter(supernode)
        types = stops.loc[stops.index.isin(supernode), 'type'].unique().tolist()

        supernodes_df.append({
            'id': i,
            'stops': list(supernode),
            'diameter': diameter,
            'types': types,
            'lat': lat,
            'lon': lon,
        })
        
    return pd.DataFrame(supernodes_df)

In [ ]:
metrics = pd.read_json(supernode_threshold_metrics_fpath, encoding='utf-8').set_index('th')

metrics.head(2)

In [ ]:
fig, axes = plt.subplots(ncols=2)
fig.set_size_inches(8, 2.5)
fig.subplots_adjust(wspace=0.3)

markers = ['o', 'v']
linestyles = ['solid', 'dashed']

for i, metric, ax in zip(range(2), ['size', 'diameter'], axes):
    color = f'C{i}'
    marker = markers[i]
    linestyle = linestyles[i]
    
    # ax.plot(metrics[f'{metric}_mean'], c=color, lw=0.8, marker='.', markersize=3)
#     ax.fill_between(
#         metrics.index,
#         metrics[f'{metric}_t1'],
#         metrics[f'{metric}_t2'],
#         color=color,
#         alpha=0.15,
#     )
    
    ax.plot(metrics[f'{metric}_max'], c=color, lw=0.8, ls=linestyle, marker=marker, markersize=3)
    
    # ax.set_title(metric)
    ax.set_xlabel('$d_0$, km')
    
axes[0].set_ylabel('supernode size')
axes[0].set_title('(a)', y=-0.34)
axes[1].set_ylabel('supernode diameter, km')
axes[1].set_title('(b)', y=-0.34)

plt.savefig('../figures/supernode_threshold_metrics.pdf', transparent=True, bbox_inches='tight')

In [8]:
supernodes = build_supernodes(0.1)

supernodes.head(2)

,id,stops,diameter,types,lat,lon
0,0,"[29058, 15366, 1687, 1688, 15385, 2080, 2081, ...",0.732963,"[bus, trolley, tram, subway]",59.85171,30.322415
1,1,"[18, 18077, 18078, 32036, 16177, 16050, 27063,...",0.909368,"[bus, tram, trolley, subway]",59.95021,30.287937


In [ ]:
supernode_sizes = supernodes['stops'].apply(len).value_counts().sort_index()

fig, (ax1, ax2) = plt.subplots(ncols=2)
fig.set_size_inches(8, 2.5)
fig.subplots_adjust(wspace=0.3)

ax1.bar(supernode_sizes.index, supernode_sizes.values, width=1)
ax1.set_xlabel('supernode size')
ax1.set_title('(a)', y=-0.32)

ax2.hist(supernodes['diameter'], color='C1', bins=25)
ax2.set_xlabel('supernode diameter, km')
ax2.set_title('(b)', y=-0.32)

plt.savefig('../figures/supernode_statistics.pdf', transparent=True, bbox_inches='tight')

In [ ]:
supernodes_json = [row.to_dict() for _, row in supernodes.iterrows()]

len(supernodes_json)

In [ ]:
supernodes_json[0]

In [ ]:
save_json(supernodes_json, supernodes_fpath)

In [ ]:
stop2supernode = {
    stop: supernode['id']
    for i, supernode in supernodes.iterrows()
    for stop in supernode['stops']
}

stops['supernode'] = stops.index.map(stop2supernode)

stops.head(2)

In [ ]:
stops_json = [row.to_dict() for _, row in stops.reset_index().iterrows()]

stops_json[0]

In [ ]:
save_json(stops_json, stops_fpath)

In [ ]:
routes = pd.read_json(routes_fpath, encoding='utf-8')

routes.head(2)

In [ ]:
def get_supernode_route(route: List[int]) -> List[int]:
    supernode_route = []
    
    for stop in route:
        supernode = stop2supernode[stop]
        
        if len(supernode_route) > 0 and supernode_route[-1] == supernode:
            continue
            
        supernode_route.append(supernode)
        
    return supernode_route


routes['supernodes'] = routes['stops'].apply(get_supernode_route)

routes.head(2)

In [ ]:
routes_json = [row.to_dict() for _, row in routes.iterrows()]

routes_json[0]

In [ ]:
save_json(routes_json, routes_fpath)

***

In [9]:
def get_supernode_radius(supernode: pd.Series) -> float:
    radius = get_earth_distances(
        stops.loc[supernode['stops'], ['lat', 'lon']].values,
        supernode[['lat', 'lon']].values.astype(float).reshape(1, -1),
    ).max()
    
    return radius


supernode_radii = pd.Series({
    i: get_supernode_radius(supernode)
    for i, supernode in supernodes.iterrows()
})

supernodes['radius'] = supernode_radii

supernodes.head(2)

,id,stops,diameter,types,lat,lon,radius
0,0,"[29058, 15366, 1687, 1688, 15385, 2080, 2081, ...",0.732963,"[bus, trolley, tram, subway]",59.85171,30.322415,0.370776
1,1,"[18, 18077, 18078, 32036, 16177, 16050, 27063,...",0.909368,"[bus, tram, trolley, subway]",59.95021,30.287937,0.472779


In [10]:
stop_coords = stops[['lon', 'lat']]

stop_coords.head(2)

,lon,lat
id,,
16396,30.494671,59.908881
16367,30.499882,59.911857


In [11]:
import warnings

warnings.filterwarnings('ignore')

In [ ]:
fig = plt.figure(figsize=(6, 5))

extent, scale, factor = vaska_area

ax = add_map_subplot(fig, 1, 1, 1, extent=extent, scale=scale)

for t in routes['type'].unique():
    type_routes = routes[routes['type'] == t].copy()
    
    color, marker, size, width, linestyle, zorder = get_node_props(t, factor)
    
    for i, route in enumerate(type_routes['stops']):
        x, y = stop_coords.loc[route].values.T
        
        label = t if i == 0 else None
        
        ax.plot(x, y, transform=ccrs.PlateCarree(), marker=marker, markersize=size,
                lw=width, ls=linestyle, c=color, label=label, zorder=zorder)
        
for i, supernode in supernodes.iterrows():
    x, y = supernode[['lon', 'lat']]
    # x, y = proj.transform_point(x, y, ax.projection)
    
    radius = supernode['radius'] + 0.01
    
    if x + radius < extent[0] or x - radius > extent[1] or y + radius < extent[2] or y - radius > extent[3]:
        continue
        
    ax.tissot(radius, lons=x, lats=y, zorder=supernode_zorder, edgecolor=supernode_color, linewidth=1, linestyle='dashed', facecolor='None')

handles, labels = ax.get_legend_handles_labels()

handles.append(plt.scatter([0], [0], marker='o', s=60, facecolor='None', color=supernode_color, linewidth=1, linestyle='dashed'))
labels.append('supernode')
    
ax.legend(handles=handles, labels=labels, loc='upper left', bbox_to_anchor=(1, 1))

plt.savefig('../figures/supernodes_map.png', dpi=250, transparent=True, bbox_inches='tight')

Geometry must be a Point or LineString
Geometry must be a Point or LineString
